Install and Import Needed Libraries

In [0]:
%pip install xgboost

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install mlflow
%pip install hyperopt

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2091d22e-9554-4bff-a0c8-b3ccd88c1704
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2091d22e-9554-4bff-a0c8-b3ccd88c1704
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2091d22e-9554-4bff-a0c8-b3ccd88c1704
  

In [0]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
 
from xgboost import XGBClassifier
 
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier
 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
 
from sklearn.metrics import average_precision_score
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
 
from hyperopt import hp, fmin, tpe, SparkTrials, STATUS_OK, space_eval
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
 
import numpy as np
 
import time

/databricks/python/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [0]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

Set up mlflow experiment in the user's personal workspace folder

In [0]:
useremail = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
experiment_name = f"/Users/{useremail}/churn"
mlflow.set_experiment(experiment_name) 

Out[3]: <Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2884970239900452', creation_time=1721583839558, experiment_id='2884970239900452', last_update_time=1721591995181, lifecycle_stage='active', name='/Users/anas.ouassou@artefact.com/churn', tags={'mlflow.experiment.sourceName': '/Users/anas.ouassou@artefact.com/churn',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'anas.ouassou@artefact.com',
 'mlflow.ownerId': '7537234187759097',
 'mlflow.sharedViewState.d195e7ebd92de2abb1dfcf380da12ad94853b9fb38fcb729aa57017f3cc8679a': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10

Data Preparation based on EDA


In [0]:
import pandas as pd

In [0]:
def datapreparation(df):
    # Define the new column names
    new_column_names = [
        "customerID", "gender", "SeniorCitizen", "Partner", "Dependents",
        "tenure", "PhoneService", "MultipleLines", "InternetService", 
        "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
        "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling",
        "PaymentMethod", "MonthlyCharges", "TotalCharges", "Churn"
    ]

    # Rename the columns
    df.columns = new_column_names

    # Drop the first row
    df = df.drop(index=0)
    df.drop(["customerID"], inplace=True, axis=1)
    
    df.TotalCharges = df.TotalCharges.replace(" ", np.nan)
    df.TotalCharges.fillna(0, inplace=True)
    df.TotalCharges = df.TotalCharges.astype(float)
    
    cols1 = ['Partner', 'Dependents', 'PaperlessBilling', 'Churn', 'PhoneService']
    for col in cols1:
        df[col] = df[col].apply(lambda x: 0 if x == "No" else 1)
    
    df.gender = df.gender.apply(lambda x: 0 if x == "Male" else 1)
    df.MultipleLines = df.MultipleLines.map({'No phone service': 0, 'No': 0, 'Yes': 1})
    
    cols2 = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    for col in cols2:
        df[col] = df[col].map({'No internet service': 0, 'No': 0, 'Yes': 1})
    
    df = pd.get_dummies(df, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)
    
    return df


Import Dataframe

In [0]:
# Create a Spark DataFrame from the table
df_spark = spark.read.table("WA_Fn_UseC__Telco_Customer_Churn_csv")

# Convert the Spark DataFrame to a Pandas DataFrame
df = df_spark.toPandas()

df = datapreparation(df)

# Display the DataFrame
df

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2884970239900495>:2
      1 # Create a Spark DataFrame from the table
----> 2 df_spark = spark.read.table("WA_Fn_UseC__Telco_Customer_Churn_csv")
      4 # Convert the Spark DataFrame to a Pandas DataFrame
      5 df = df_spark.toPandas()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:473, in DataFrameReader.table(self, tableName)
    439 def table(self, tableName: str) -> "DataFrame":
    440     """Returns the specified table as a :class:`DataFr

Some Feature engineering

In [0]:
# Ensure that SeniorCitizen, tenure, and MonthlyCharges are numeric
df['SeniorCitizen'] = pd.to_numeric(df['SeniorCitizen'], errors='coerce')
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')

X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure all columns in X_train and X_test are numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Check the data types
print(X_train.dtypes)
print(X_test.dtypes)

Logistic Regression

In [0]:
# Ensure that SeniorCitizen, tenure, and MonthlyCharges are numeric
df['SeniorCitizen'] = pd.to_numeric(df['SeniorCitizen'], errors='coerce')
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')

X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure all columns in X_train and X_test are numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Check the data types
print(X_train.dtypes)
print(X_test.dtypes)



Define weights for the XGBoost model 

In [0]:
weights = compute_class_weight(
  'balanced', 
  classes=np.unique(y_train), 
  y=y_train
  )
 
weights

In [0]:
def evaluate_model(hyperopt_params):
  
  # accesss replicated input data
  X_train_input = X_train_broadcast.value
  y_train_input = y_train_broadcast.value
  X_test_input = X_test_broadcast.value
  y_test_input = y_test_broadcast.value  
  
  # configure model parameters
  params = hyperopt_params
  
  if 'max_depth' in params: params['max_depth']=int(params['max_depth'])   # hyperopt supplies values as float but must be int
  if 'min_child_weight' in params: params['min_child_weight']=int(params['min_child_weight']) # hyperopt supplies values as float but must be int
  if 'max_delta_step' in params: params['max_delta_step']=int(params['max_delta_step']) # hyperopt supplies values as float but must be int
  # all other hyperparameters are taken as given by hyperopt
  
  # params['tree_method']='gpu_hist'      # settings for running on GPU
  # params['predictor']='gpu_predictor'   # settings for running on GPU
  
  params['tree_method']='hist'      # settings for running on CPU
  params['predictor']='cpu_predictor'   # settings for running on CPU
  
  # instantiate model with parameters
  model = XGBClassifier(**params)
  
  # train
  model.fit(X_train_input, y_train_input)
  
  # predict
  y_prob = model.predict_proba(X_test_input)
  
  # score
  model_ap = average_precision_score(y_test_input, y_prob[:,1])
  mlflow.log_metric('avg precision', model_ap)  # record actual metric with mlflow run
  
  # invert metric for hyperopt
  loss = -1 * model_ap  
  
  # return results
  return {'loss': loss, 'status': STATUS_OK}

In [0]:
X_train_broadcast = sc.broadcast(X_train)
X_test_broadcast = sc.broadcast(X_test)
y_train_broadcast = sc.broadcast(y_train)
y_test_broadcast = sc.broadcast(y_test)

In [0]:
scale = weights[1]/weights[0]
 
# define hyperopt search space
search_space = {
    'max_depth' : hp.quniform('max_depth', 1, 30, 1)                                  # depth of trees (preference is for shallow trees or even stumps (max_depth=1))
    ,'learning_rate' : hp.loguniform('learning_rate', np.log(0.01), np.log(0.40))     # learning rate for XGBoost
    ,'gamma': hp.quniform('gamma', 0.0, 1.0, 0.001)                                   # minimum loss reduction required to make a further partition on a leaf node
    ,'min_child_weight' : hp.quniform('min_child_weight', 1, 20, 1)                   # minimum number of instances per node
    ,'subsample' : hp.loguniform('subsample', np.log(0.1), np.log(1.0))               # random selection of rows for training,
    ,'colsample_bytree' : hp.loguniform('colsample_bytree', np.log(0.1), np.log(1.0)) # proportion of columns to use per tree
    ,'colsample_bylevel': hp.loguniform('colsample_bylevel', np.log(0.1), np.log(1.0))# proportion of columns to use per level
    ,'colsample_bynode' : hp.loguniform('colsample_bynode', np.log(0.1), np.log(1.0)) # proportion of columns to use per node
    ,'scale_pos_weight' : hp.loguniform('scale_pos_weight', np.log(scale), np.log(scale * 10))   # weight to assign positive label to manage imbalance
    }

In [0]:
# perform evaluation
with mlflow.start_run(run_name='HGB'):
  argmin = fmin(
    fn=evaluate_model,
    space=search_space,
    algo=tpe.suggest,  # algorithm controlling how hyperopt navigates the search space
    max_evals=250,
    # trials=SparkTrials(parallelism=2), # THIS VALUE IS ALIGNED WITH THE NUMBER OF WORKERS IN MY GPU-ENABLED CLUSTER (guidance differs for CPU-based clusters)
    verbose=True
    )

In [0]:
# release the broadcast datasets
X_train_broadcast.unpersist()
X_test_broadcast.unpersist()
y_train_broadcast.unpersist()
y_test_broadcast.unpersist()

In [0]:
 space_eval(search_space, argmin)

In [0]:
# define list to hold run ids for later retrieval
run_ids = []

# train model with optimal settings 
with mlflow.start_run(run_name='XGB Final Model') as run:
  
  # capture run info for later use
  run_id = run.info.run_id
  run_name = run.data.tags['mlflow.runName']
  run_ids += [(run_name, run_id)]
   
  # configure params
  params = space_eval(search_space, argmin)
  if 'max_depth' in params: params['max_depth']=int(params['max_depth'])       
  if 'min_child_weight' in params: params['min_child_weight']=int(params['min_child_weight'])
  if 'max_delta_step' in params: params['max_delta_step']=int(params['max_delta_step'])
  if 'scale_pos_weight' in params: params['scale_pos_weight']=int(params['scale_pos_weight'])    
  params['tree_method']='hist'        # modified for CPU deployment
  params['predictor']='cpu_predictor' # modified for CPU deployment
  mlflow.log_params(params)
  
  # train
  model = XGBClassifier(**params)
  model.fit(X_train, y_train)
  mlflow.sklearn.log_model(model, 'model')  # persist model with mlflow
  
  # predict
  y_prob = model.predict_proba(X_test)
  
  # score
  model_ap = average_precision_score(y_test, y_prob[:,1])
  mlflow.log_metric('avg precision', model_ap)
  
  print('Model logged under run_id "{0}" with AP score of {1:.5f}'.format(run_id, model_ap))

In [0]:
print(y_prob[:,1])